In [ ]:
!pip install -U selenium

     |████████████████████████████████| 911kB 3.9MB/s 


In [ ]:
import re
import pandas as pd
from bs4 import BeautifulSoup
import lxml
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import requests
import time
import numpy as np
import itertools
import sys

In [ ]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [ ]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument(f'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36')

In [ ]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)
url = "https://www.tokopedia.com/search?st=product&q=masker%20n95&navsource=home"
driver.get(url)
elements_small = driver.find_elements_by_class_name('css-18c4yhp')
driver.execute_script("return arguments[0].scrollIntoView();", elements_small[-1])
time.sleep(3)

soup = BeautifulSoup(driver.page_source,'lxml')
base_div = soup.find_all('div',{'class':'css-jza1fo'})
name = []
price = []
rating = []
for i in base_div:
    nam = i.find_all('div',class_='css-18c4yhp')
    pri = i.find_all('div',class_='css-rhd610')
    rat = i.find_all('span',class_='css-etd83i')
    for j,k,l in zip(nam,pri,rat):
        name.append(j.text)
        price.append(k.text)
        if not l:
            l = 'Null'
        else:
            l = l.text
        rating.append(l)

#price_class = base_div.find_all('div',class_='css-rhd610')
#storeLoc_class = base_div.find_all('span',class_='css-4pwgpi')
#rating_data = base_div.find_all('span',class_='css-etd83i')

In [ ]:
print(len(name))
print(len(price))
print(len(rating))

79
79
79


In [ ]:
def scrape_tokopedia(url, export_csv=False, csv_name=None, csv_path=None, g_colabs=True, source=False):
    import os
    chrome_options = webdriver.ChromeOptions()
    #run chrome w/o GUI
    chrome_options.add_argument('--headless')
    #no sandbox feature ONLY for trusted sites
    chrome_options.add_argument('--no-sandbox')
    #disable use RAM for temp storage
    chrome_options.add_argument('--disable-dev-shm-usage')
    #header argument
    chrome_options.add_argument(f'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36')
    
    if g_colabs == True:
        driver = webdriver.Chrome('chromedriver',options= chrome_options)
    else:
        driver = webdriver.Chrome()
    
    #request source code
    driver.get(url)
    #initial name before scroll
    elements1 = driver.find_elements_by_class_name('css-18c4yhp')
    #simulate scroll down
    driver.execute_script("return arguments[0].scrollIntoView();", elements1[-1])
    time.sleep(3)
    #all data should be complete in the first page
    source = BeautifulSoup(driver.page_source,features='lxml')
        
    #base source code
    base_div = source.find_all('div',class_='css-jza1fo')
    name = []
    price = []
    rating = []
    sold = []
    for i in base_div:
        #name container
        nam = i.find_all('div',class_='css-18c4yhp')
        #price container
        pri = i.find_all('div',class_='css-rhd610')
        #rating container
        rat = i.find_all('span',class_='css-etd83i')
        #sold info container
        sol = i.find_all('span',class_='css-1kgbcz0')
        for j,k,l,m in zip(nam,pri,rat,sol):
            name.append(j.text)
            price.append(k.text)
            if not l and m:
                l = 'Null'
                m = 'Null'
            else:
                l = l.text
                m = m.text
            rating.append(l)
            sold.append(m)
        
    df = pd.DataFrame({'Nama Produk':np.array(name),
                    'Harga_IDR':np.array(price),
                    'Rating':np.array(rating),
                    'Terjual':np.array(sold)})
        
    if export_csv == True:
        if csv_path is None:
            #current working directory
            temp_path = os.getcwd()
            df.to_csv(str(temp_path)+'/'+str(csv_name),index_label=False)          
        else:
            df.to_csv(csv_path,index_label=False)
    
    if source == True:
        return source,df
    else:
        return df

In [ ]:
df = scrape_tokopedia("https://www.tokopedia.com/search?st=product&q=masker%20n95&navsource=home",export_csv=True,csv_name='masker n95.csv')

In [ ]:
df.head()

,Nama Produk,Harga_IDR,Rating,Terjual
0,Masker 3M N95 1870+ AURA Original 1 Box isi 20...,Rp355.000,4.8,Terjual 277
1,masker n95 3m respirator valve 9501V /9001V or...,Rp17.000,4.9,Terjual 13 rb
2,MASKER N95/ 3M / 1860 USA - USA,Rp175.000,4.7,Terjual 233
3,masker 3M 8210/masker N95,Rp240.000,4.9,"Terjual 1,0 rb"
4,Masker KN95 Masker 5 Ply N95 Masker KN 95 Mask...,Rp1.000,4.9,Terjual 40 rb


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Nama Produk  79 non-null     object
 1   Harga_IDR    79 non-null     object
 2   Rating       79 non-null     object
 3   Terjual      79 non-null     object
dtypes: object(4)
memory usage: 2.6+ KB


In [ ]:
for i in range(2,25):
    data = scrape_tokopedia(f'https://www.tokopedia.com/search?navsource=home&page={i}&q=masker%20n95&st=product')
    df = df.append(data,ignore_index=False).reset_index(drop=True)

In [ ]:
df

,Nama Produk,Harga_IDR,Rating,Terjual
0,Masker 3M N95 1870+ AURA Original 1 Box isi 20...,Rp355.000,4.8,Terjual 277
1,masker n95 3m respirator valve 9501V /9001V or...,Rp17.000,4.9,Terjual 13 rb
2,MASKER N95/ 3M / 1860 USA - USA,Rp175.000,4.7,Terjual 233
3,masker 3M 8210/masker N95,Rp240.000,4.9,"Terjual 1,0 rb"
4,Masker KN95 Masker 5 Ply N95 Masker KN 95 Mask...,Rp1.000,4.9,Terjual 40 rb
...,...,...,...,...
602,Masker KN95 Mas N95 Surgical 5ply KN N 95 pack...,Rp3.599,4.8,Terjual 33
603,MASKER 3M N95 9010 PUTIH MURAH,Rp55.000,5.0,Terjual 2
604,Filter Masker 3M 5N11CN Niosh N95 per Box isi ...,Rp90.000,5.0,Terjual 47
605,Masker N95 3M 9501V+ Original | Earloop Eceran...,Rp18.000,5.0,Terjual 245
